In [158]:
from path import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import RandomOverSampler

In [18]:
red_data = Path('all_wines.csv')
red_df = pd.read_csv(red_data)
red_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,sulphates,alcohol,ph,quality,id,color
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1,red
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,2,red
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,3,red
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,4,red
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,5,red


#### Filter to red wines only

In [21]:
red_df = red_df[red_df["color"] == "red"]
red_df.shape

(1599, 14)

#### Categorize quality to >=7 as Good and < 7 and Not good

In [22]:
red_df['quality_2'] = (red_df['quality'] >=  7).astype(int)

In [26]:
red_df.head(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,sulphates,alcohol,ph,quality,id,color,quality_2
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,1,red,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,2,red,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,3,red,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,4,red,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,5,red,0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5,6,red,0
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5,7,red,0
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7,8,red,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7,9,red,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5,10,red,0


#### Create Target and Features

In [27]:
y = red_df["quality_2"]
X = red_df.drop(columns=["quality","quality_2","id","color"])

In [28]:
X.head(10)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,sulphates,alcohol,ph
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5


In [31]:
y.value_counts()

0    1382
1     217
Name: quality_2, dtype: int64

#### Scale the data

In [37]:
data_scaler = StandardScaler()

In [40]:
X_scaled = data_scaler.fit_transform(X)
# X_scaled

#### Split the data into test and train

In [82]:
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=7)

In [83]:
y_train.value_counts()

0    1033
1     166
Name: quality_2, dtype: int64

In [84]:
X_scaled.shape

(1599, 11)

## Logisitic model

In [85]:
model1 = LogisticRegression(solver='lbfgs',random_state=7)

In [86]:
model1.fit(X_train, y_train)

LogisticRegression(random_state=1)

#### Model predictions

In [87]:
y_pred = model1.predict(X_test)

In [88]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[329  20]
 [ 29  22]]


In [89]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       349
           1       0.52      0.43      0.47        51

    accuracy                           0.88       400
   macro avg       0.72      0.69      0.70       400
weighted avg       0.87      0.88      0.87       400



## SVM

In [102]:
# Linear kernel did not work
model2 = SVC(kernel='rbf',random_state=7)

In [103]:
model2.fit(X_train, y_train)

SVC()

In [104]:
y_pred = model2.predict(X_test)

In [105]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[336  13]
 [ 34  17]]


In [106]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.96      0.93       349
           1       0.57      0.33      0.42        51

    accuracy                           0.88       400
   macro avg       0.74      0.65      0.68       400
weighted avg       0.86      0.88      0.87       400



In [107]:
accuracy_score(y_test, y_pred)

0.8825

## Decision tree

In [119]:
model3 = tree.DecisionTreeClassifier(random_state=7)

In [120]:
model3 = model3.fit(X_train, y_train)

In [121]:
y_pred = model3.predict(X_test)

In [122]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[305  44]
 [ 24  27]]


In [123]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.87      0.90       349
           1       0.38      0.53      0.44        51

    accuracy                           0.83       400
   macro avg       0.65      0.70      0.67       400
weighted avg       0.86      0.83      0.84       400



In [124]:
accuracy_score(y_test, y_pred)

0.83

In [140]:
importance = model3.feature_importances_
columns = X.columns
import_df = pd.DataFrame({'feature':columns, 'importance':importance})
import_df = import_df.sort_values(["importance"], ascending=False)
import_df

,feature,importance
10,ph,0.249580
8,sulphates,0.136080
9,alcohol,0.114116
0,fixed_acidity,0.087731
1,volatile_acidity,0.081758
5,free_sulfur_dioxide,0.081022
2,citric_acid,0.078951
3,residual_sugar,0.045753
7,density,0.045384
6,total_sulfur_dioxide,0.040393


## Random Forest Classifier

In [126]:
model4 = RandomForestClassifier(n_estimators=128,random_state=7) 

In [127]:
model4 = model4.fit(X_train, y_train)

In [128]:
y_pred = model4.predict(X_test)

In [129]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[333  16]
 [ 25  26]]


In [130]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       349
           1       0.62      0.51      0.56        51

    accuracy                           0.90       400
   macro avg       0.77      0.73      0.75       400
weighted avg       0.89      0.90      0.89       400



In [131]:
accuracy_score(y_test, y_pred)

0.8975

In [139]:
importance = model4.feature_importances_
columns = X.columns
import_df = pd.DataFrame({'feature':columns, 'importance':importance})
import_df = import_df.sort_values(["importance"], ascending=False)
import_df

,feature,importance
10,ph,0.169798
1,volatile_acidity,0.120064
9,alcohol,0.112396
7,density,0.092389
6,total_sulfur_dioxide,0.083003
2,citric_acid,0.079479
0,fixed_acidity,0.075408
4,chlorides,0.072150
3,residual_sugar,0.071893
8,sulphates,0.062759


## Gradient Boosted Tree

In [150]:
# Create a classifier object
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    classifier = GradientBoostingClassifier(n_estimators=20,
                                            learning_rate=learning_rate,
                                            max_features=5,
                                            max_depth=3,
                                            random_state=0)

    # Fit the model
    classifier.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)

    # Score the model
    print("Accuracy score (training): {0:.3f}".format(
        classifier.score(
            X_train,
            y_train)))
    print("Accuracy score (validation): {0:.3f}".format(
        classifier.score(
            X_test,
            y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.887
Accuracy score (validation): 0.880

Learning rate:  0.1
Accuracy score (training): 0.914
Accuracy score (validation): 0.882

Learning rate:  0.25
Accuracy score (training): 0.944
Accuracy score (validation): 0.882

Learning rate:  0.5
Accuracy score (training): 0.959
Accuracy score (validation): 0.892

Learning rate:  0.75
Accuracy score (training): 0.973
Accuracy score (validation): 0.865

Learning rate:  1
Accuracy score (training): 0.973
Accuracy score (validation): 0.858



In [151]:
model5 = GradientBoostingClassifier(n_estimators=20,
   learning_rate=0.5, max_features=10, max_depth=3, random_state=7)

In [152]:
model5 = model5.fit(X_train, y_train)

In [153]:
y_pred = model5.predict(X_test)

In [154]:
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

[[332  17]
 [ 29  22]]


In [155]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.92      0.95      0.94       349
           1       0.56      0.43      0.49        51

    accuracy                           0.89       400
   macro avg       0.74      0.69      0.71       400
weighted avg       0.87      0.89      0.88       400



In [156]:
accuracy_score(y_test, y_pred)

0.885

In [157]:
importance = model5.feature_importances_
columns = X.columns
import_df = pd.DataFrame({'feature':columns, 'importance':importance})
import_df = import_df.sort_values(["importance"], ascending=False)
import_df

,feature,importance
10,ph,0.306515
6,total_sulfur_dioxide,0.131908
9,alcohol,0.126080
1,volatile_acidity,0.122331
5,free_sulfur_dioxide,0.097373
3,residual_sugar,0.054741
4,chlorides,0.045010
2,citric_acid,0.040504
7,density,0.038264
8,sulphates,0.018796


## Random Over Sampling